In [75]:
!pip install numpy moabb

In [26]:
import moabb
import numpy as np
import mne
import torch
import torch.nn as nn
import torch.optim as optim
from moabb.datasets import MAMEM3

from moabb.paradigms import SSVEP
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from scipy.fftpack import fft

In [27]:
fmin = 6  # 최소 주파수 (6Hz)
fmax = 14  # 최대 주파수 (14Hz)
paradigm = SSVEP(fmin=fmin, fmax=fmax, resample=128)

# ✅ MAMEM3 데이터셋 불러오기 (첫 번째 참가자)
dataset = MAMEM3()
X, y, metadata = paradigm.get_data(dataset=dataset, subjects=[1])

print(f"✅ 데이터 로드 완료: X shape: {X.shape}, y shape: {y.shape}")

Choosing the first None classes from all possible events


✅ 데이터 로드 완료: X shape: (80, 14, 385), y shape: (80,)


C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 4 s (baseline off), ~447 KiB, data loaded,
 '6.66': 3
 '7.50': 0
 '8.57': 2
 '10.00': 2
 '12.00': 3>
  warn(f"warnEpochs {epochs}")
C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 4 s (baseline off), ~447 KiB, data loaded,
 '6.66': 1
 '7.50': 0
 '8.57': 3
 '10.00': 4
 '12.00': 2>
  warn(f"warnEpochs {epochs}")
C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not f

In [28]:
# ✅ FFT를 이용한 주파수 특징 추출 함수
def extract_fft_features(X, sfreq=128):
    num_samples, num_channels, num_timepoints = X.shape
    fft_values = np.abs(fft(X, axis=2))  # FFT 적용 (절대값)
    
    # 관심 주파수 대역 선택 (6~14Hz)
    freqs = np.fft.fftfreq(num_timepoints, d=1/sfreq)
    freq_mask = (freqs >= fmin) & (freqs <= fmax)
    
    return fft_values[:, :, freq_mask]

In [29]:
# ✅ FFT 변환 적용
X_fft = extract_fft_features(X)
X_fft = np.expand_dims(X_fft, axis=1)  # (batch, 1, channels, time)
print(f"✅ FFT 변환 완료: X_fft shape: {X_fft.shape}")

# ✅ 데이터 분할 (훈련: 80%, 테스트: 20%)
X_train, X_test, y_train, y_test = train_test_split(X_fft, y, test_size=0.2, random_state=42)

✅ FFT 변환 완료: X_fft shape: (80, 1, 14, 24)


In [30]:
# ✅ PyTorch 데이터셋 변환

# X 데이터는 float32 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# ✅ 클래스 매핑을 위한 고유한 주파수 목록 (MAMEM3 실험에서 사용된 5개 주파수)
unique_freqs = np.unique(y_train)  # y_train과 y_test에 있는 고유한 주파수 값

# ✅ 주파수를 정수 인덱스로 매핑 (예: 6.66Hz → 0, 7.50Hz → 1 ...)
freq_to_index = {freq: idx for idx, freq in enumerate(unique_freqs)}

# ✅ 주파수를 정수 인덱스로 변환
y_train = np.array([freq_to_index[freq] for freq in y_train])
y_test = np.array([freq_to_index[freq] for freq in y_test])

# ✅ PyTorch Tensor 변환
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# ✅ TensorDataset 및 DataLoader 구성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("✅ 데이터 변환 완료!")

✅ 데이터 변환 완료!


In [31]:
# ✅ EEGNet 모델 정의
class EEGNet(nn.Module):
    def __init__(self, num_classes=5):
        super(EEGNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, (1, 64), padding=(0, 32))
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, (14, 1), groups=16)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.pool = nn.AvgPool2d((1, 4))
        self.fc1 = nn.Linear(32 * (X_train.shape[3]//4), num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        return x

In [32]:
# ✅ 모델 초기화
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EEGNet(num_classes=len(np.unique(y))).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [33]:

# ✅ 모델 학습 루프
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

print("✅ 학습 완료!")

Epoch 1/20, Loss: 1.4311197400093079
Epoch 2/20, Loss: 1.3239556849002838
Epoch 3/20, Loss: 1.2785580158233643
Epoch 4/20, Loss: 1.2436258494853973
Epoch 5/20, Loss: 1.216487169265747
Epoch 6/20, Loss: 1.171218603849411
Epoch 7/20, Loss: 1.1580175161361694
Epoch 8/20, Loss: 1.0895709991455078
Epoch 9/20, Loss: 1.0646306574344635
Epoch 10/20, Loss: 1.033623918890953
Epoch 11/20, Loss: 0.9892938435077667
Epoch 12/20, Loss: 0.9503342658281326
Epoch 13/20, Loss: 0.929772362112999
Epoch 14/20, Loss: 0.8856446743011475
Epoch 15/20, Loss: 0.876471221446991
Epoch 16/20, Loss: 0.8340339660644531
Epoch 17/20, Loss: 0.7940598130226135
Epoch 18/20, Loss: 0.7904439866542816
Epoch 19/20, Loss: 0.7644221782684326
Epoch 20/20, Loss: 0.7252548784017563
✅ 학습 완료!


In [34]:
# ✅ 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"✅ 모델 정확도: {accuracy:.2f}%")

✅ 모델 정확도: 75.00%
